In [8]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/s4_happy/happy_s4


In [1]:
import pandas as pd

In [2]:
from datasets import load_metric

In [12]:
from datasets import load_dataset

In [16]:
from happy_s4.data.lit_data import HFLitDataArgs, HFLitDataModule

In [50]:
from pathlib import Path

In [135]:
outputs_data = "outputs/s4-exp-1/superglue/rte"

In [136]:
with open(Path(outputs_data) / "best_model_path.txt", "r+") as file:
    best_model_path = file.read().split()[0]

In [137]:
metric = load_metric(*outputs_data.split("/")[-2:])

FileNotFoundError: Couldn't find a metric script at /mnt/data4/haryoaw_workspace/projects/2021_2/s4_happy/happy_s4/superglue/superglue.py. Metric 'superglue' doesn't exist on the Hugging Face Hub either.

In [138]:
metric = load_metric("super_glue", "rte")

In [139]:
task_name = '/'.join(outputs_data.split("/")[-2:])

In [140]:
task_name

'superglue/rte'

In [141]:
from happy_s4.model.lit_model import LitS4

In [142]:
model = LitS4.load_from_checkpoint(best_model_path)

{'dt': True, 'A': True, 'P': True, 'B': True}
{'dt': True, 'A': True, 'P': True, 'B': True}
{'dt': True, 'A': True, 'P': True, 'B': True}
{'dt': True, 'A': True, 'P': True, 'B': True}


In [144]:
model = model.eval()

In [145]:
data_inp_list = {
    "superglue/rte": {
        "data_name": "super_glue",
        "data_subset": "rte",
        "column_train": ["premise", "hypothesis"],
        "column_label": "label"
    },
    "glue/mrpc": {
        "data_name": "glue",
        "data_subset": "mrpc",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
    "glue/sst2": {
        "data_name": "glue",
        "data_subset": "sst2",
        "column_train": ["sentence"],
        "column_label": "label"
    },
    "glue/qqp": {
        "data_name": "glue",
        "data_subset": "qqp",
        "column_train": ["question1", "question2"],
        "column_label": "label"
    },
    "glue/qnli": {
        "data_name": "glue",
        "data_subset": "qnli",
        "column_train": ["question", "sentence"],
        "column_label": "label"
    },
    "glue/rte": {
        "data_name": "glue",
        "data_subset": "rte",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
    "glue/wnli": {
        "data_name": "glue",
        "data_subset": "wnli",
        "column_train": ["sentence1", "sentence2"],
        "column_label": "label"
    },
}

In [146]:
lit_data_args = HFLitDataArgs(split_pattern="", **data_inp_list[task_name])
dm = HFLitDataModule(lit_data_args)
dm.setup()

Reusing dataset super_glue (/mnt/data1/hf_dataset_cache/super_glue/rte/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/super_glue/rte/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7/cache-dbf04bb0e3c3b98f.arrow
Reusing dataset super_glue (/mnt/data1/hf_dataset_cache/super_glue/rte/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/super_glue/rte/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7/cache-ce0608bb7829607b.arrow


In [147]:
dl = dm.val_dataloader()

In [148]:
from tqdm import tqdm

In [149]:
import torch

with torch.no_grad():
    for batch in tqdm(dl):
        _, logits = model(input_ids=batch['input_ids'], labels=batch['label'], attention_masks=batch['attention_masks'])
        pred_batch = logits.argmax(dim=1).numpy().tolist()
        gt_batch = batch['label'].numpy().tolist()
        metric.add_batch(predictions=pred_batch, references=gt_batch)

100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  4.70it/s]


In [150]:
metric.compute()

{'accuracy': 0.5342960288808665}